In [2]:
# Initialize FRED API with your API key
FRED_API_KEY = 'a6b717ac7380adb4b4e8dee590c8bc28'

[*********************100%***********************]  667 of 667 completed


KeyError: 'NVO'

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from ta.momentum import RSIIndicator
from ta.trend import MACD
from smtplib import SMTP
from datetime import datetime, timedelta
from fredapi import Fred

# Initialize FRED API with your API key
fred = Fred(api_key=FRED_API_KEY)

# Load biotech tickers
def get_biotech_tickers():
    df_biotech_tickers = pd.read_csv("biotech_tickers.csv")
    return df_biotech_tickers['Ticker'].tolist()

# Fetch market data
def fetch_data(tickers, start_date, end_date):
    return yf.download(tickers, start=start_date, end=end_date)

# Calculate RSI
def calculate_rsi(data, period=14):
    rsi = RSIIndicator(data['Close'], window=period)
    return rsi.rsi()

# Calculate MACD
def calculate_macd(data):
    macd = MACD(data['Close'])
    return macd.macd(), macd.macd_signal()

# Fetch macroeconomic data from FRED
def fetch_macro_data():
    gdp = fred.get_series('GDP', observation_start='2020-01-01')
    inflation = fred.get_series('CPIAUCSL', observation_start='2020-01-01')  # Consumer Price Index for All Urban Consumers: All Items in U.S. City Average
    return {'GDP': gdp.iloc[-1], 'Inflation': inflation.pct_change().iloc[-1] * 100}

# Generate signals
def generate_signals(data, rsi_threshold=30):
    signals = data[(data['RSI'] < rsi_threshold) & (data['MACD'] > data['MACD_Signal'])]
    return signals

# Backtest strategy
def backtest_strategy(data):
    # Implement backtesting logic here using historical data and signals
    pass

# Send email alert
def send_email_alert(subject, body):
    with SMTP('smtp.example.com') as smtp:
        smtp.login('user@example.com', 'password')
        message = f'Subject: {subject}\n\n{body}'
        smtp.sendmail('from@example.com', 'to@example.com', message)

# Main function to run analysis
def main():
    tickers = get_biotech_tickers()
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*4)  # 4 years of data
    
    data = fetch_data(tickers + ['XBI'], start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
    
    # Calculate RSI and MACD for each ticker
    for ticker in tickers:
        data[ticker, 'RSI'] = calculate_rsi(data[ticker])
        data[ticker, 'MACD'], data[ticker, 'MACD_Signal'] = calculate_macd(data[ticker])
    
    # Fetch macroeconomic data from FRED
    macro_data = fetch_macro_data()
    
    # Generate signals based on conditions and macroeconomic context
    signals = generate_signals(data)
    
    # Backtest strategy with macro context
    backtest_strategy(data)
    
    # Automate and send alerts if conditions are met
    if not signals.empty:
        send_email_alert('Biotech Signal Alert', f'Signals detected with macro context: {macro_data}')

if __name__ == "__main__":
    main()